In [58]:
from dotenv import load_dotenv
import os
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
load_dotenv()

In [59]:
options  = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)

In [60]:
url = 'https://sede.madrid.es/portal/site/tramites/menuitem.62876cb64654a55e2dbd7003a8a409a0/?vgnextoid=e3b0234be7924710VgnVCM2000001f4a900aRCRD&vgnextchannel=b59637c190180210VgnVCM100000c90da8c0RCRD&vgnextfmt=pd'
driver.get(url)

In [61]:
rechazar = driver.find_element(By.XPATH, '//*[@id="iam-cookie-control-modal-action-secondary"]')
rechazar.click()

In [62]:
en_linea = driver.find_element(By.XPATH, '//*[@id="verTodas"]/div/div[1]/p/a')
en_linea.click()

In [63]:
driver.close()
driver.switch_to.window(driver.window_handles[0])

usuario_contrasena = driver.find_element(By.XPATH, '//*[@id="ContentFixedSection_uSecciones_divSections"]/section[1]/div[2]/div/div/div[2]/article[1]')
usuario_contrasena.click()

In [64]:
acceso = driver.find_element(By.XPATH, '//*[@id="acceso_pass"]')
acceso.click()

In [65]:
usuario = driver.find_element(By.XPATH, '//*[@id="correoelectronico"]')
usuario.send_keys(os.environ.get('user'))

contrasena = driver.find_element(By.XPATH, '//*[@id="contrasenia"]')
contrasena.send_keys(os.environ.get('pass'))
contrasena.submit()

In [66]:
actividades_dia_centro = driver.find_element(By.XPATH, '//*[@id="ContentFixedSection_uSecciones_divSections"]/section[3]/div[2]/div/div/div[2]/article[1]/div')
actividades_dia_centro.click()

In [67]:
disponibilidad = driver.find_element(By.XPATH, '//*[@id="ContentFixedSection_uAltaEventos_uCentrosSeleccionar_availability_filter_on"]')
disponibilidad.click()

In [68]:
time.sleep(1)
centros = driver.find_elements(By.XPATH, "//li[contains(@class, 'media pull-left')]")

In [69]:
columnas = ["Centro", "Dirección", "Actividad", "Horario", "Rango de edad", "Duración"]
df = pd.DataFrame(columns=columnas)

for i in range(len(centros)):
    centro = centros[i]
    time.sleep(3)
    centro.click()
    time.sleep(3)

    nombre = driver.find_element(By.XPATH, '//*[@id="ContentFixedSection_uAltaEventos_divFacility"]/div/h2')
    direccion = driver.find_element(By.XPATH, '//*[@id="ContentFixedSection_uAltaEventos_divFacility"]/div/div[2]')
    elementos_h4 = driver.find_elements(By.XPATH, '//div[@class="collapse in"]//h4')

    actividades_con_info_adicional = []

    rango_edad_actual = None
    duracion_actual = None

    patron_rango_edad_flexible = re.compile(r"(?i)\b(?:De\s*)?(\d+)\s*a\s*(\d+)\s*años\b")
    patron_rango_edad_apartirde = re.compile(r"(?i)\bA\s*partir\s+de\s+(\d+)\s+años\b")
    patron_duracion = r"(\d+)\s*[´'`]"
    patron_hora = r"\d{1,2}:\d{2}"

    for elemento in elementos_h4:
        texto = elemento.text
        
        if re.match(patron_hora, texto):
            actividades_con_info_adicional[-1][1].append(texto)
        else:
            titulo_actual = texto
            rango_edad_actual = None
            duracion_actual = None
            elementos_p = elemento.find_elements(By.XPATH, './following-sibling::p')
            
            for elem_p in elementos_p:
                texto_p = elem_p.text
                resultado_rango_edad = re.search(patron_rango_edad_flexible, texto_p)
                
                if resultado_rango_edad:
                    rango_edad_actual = f"De {resultado_rango_edad.group(1)} a {resultado_rango_edad.group(2)} años"

                else:
                    resultado_rango_edad_apartirde = re.search(patron_rango_edad_apartirde, texto_p)
                    if resultado_rango_edad_apartirde:
                        rango_edad_actual = f"A partir de {resultado_rango_edad_apartirde.group(1)} años"
                        
                resultado_duracion = re.search(patron_duracion, texto_p)
                if resultado_duracion:
                    duracion_actual = f"{resultado_duracion.group(1)}'"

            actividades_con_info_adicional.append([titulo_actual, [], rango_edad_actual, duracion_actual])

    for actividad in actividades_con_info_adicional:
        titulo = actividad[0]
        horas = actividad[1]
        rango_edad = actividad[2]
        duracion = actividad[3]
        df.loc[len(df)] = [nombre.text, direccion.text, titulo, horas, rango_edad, duracion]

    back = driver.find_element(By.XPATH, '//*[@id="ContentFixedSection_uAltaEventos_divFacility"]/div/div[1]/span')
    back.click()

df

,Centro,Dirección,Actividad,Horario,Rango de edad,Duración
0,Cubo Gym Las Tablas,"Calle Fromista Esq Calle Castiello de Jaca, 28...",Cubo Gym Cross Training/Funcional,"[18:00, 19:00, 20:00, 21:00]",De 15 a 64 años,60'
1,Cubo Gym Madrid Río,"Paseo Bajo de la Virgen del Puerto, 28005, Madrid",Cubo Gym Cross Training/Funcional,"[18:00, 19:00, 20:00, 21:00]",De 15 a 64 años,45'
2,Cubo Gym Retiro,"Pª de Fernán Núñez, (en ""Capricho Castillete"")...",Cubo Gym Cross Training/Funcional,"[18:00, 19:00, 20:00, 21:00]",De 15 a 64 años,60'


In [ ]:
# Ya está automatizado el scraping, solo faltaría editar el dataframe y corregir algunos datos, como añadir el teléfono o distrito, fecha
# También hay que automatizar la subida de datos a la base de datos.

# Pruebas